# 📘 Semi-Implicit PIC Solver in Electromagnetic Fields


This project implements a **semi-implicit Particle-in-Cell (PIC) solver** for simulating
particle motion in **electric** and **magnetic fields**, inspired by the formulations from
**Vu & Brackbill (1982)** and **Lapenta (2006)**.



# 🧩 Core Algorithm Steps

## Initialization
- Simulation Paramter 3D
- Border 25 x 25 x 25
- Grid Points 10 x 10 x 10
- Particle per Cell [NPpCell] 20
- Implicit : Theta 0.5 <br/>
### Grid
- Grid is saved as Mesh (X,Y,Z) np.meshgrid   so you get Xpositon with X[x][y][z] <br/>
- The Cmd :"Indexing i,j" Is for the Order of Dimension x,y,z  if you choose x,y it whould switches bc it just does [arrayx*y]*z  
- Grid Point Has Fields
- Every Grid Point 1 Electric Field ($\theta$) and Magnetic Field
- Also density $\rho$ <br/>

#### Computational Particel
- Group Of particles it Represent(Constant)
- with Custum Shape Function Exists between the Grid Points
- Velocity,Force,Elekctric Field,Magnetic Field <br/>
- Are all 3 Dimensional First Index (x,y,z) <br/>
- Second Index Particel Index  Total=(NPpCell*VolumeGridpoints(100)) <br/>


# Developing Step
- **Deposition + Moment Gathering:** Get from Particles to Grid (Computational Particles Density)
- **Evolve E**  
- **Convert to Particle** Grid Fields to Fp (Calculate Fields for individual Particel)
 
## Itereation Particle Mover (Semi-Implicit)
**Iter over i Start for i=0 with:** <br/>

**$\overline{x^{n}}_0=x^{n}$** <br/>

**$B^{n}(\overline{x}_i)$** <br/>

Than do Calculation to develop:
- $E^{n+\theta} (\overline{x}_i)$ <br/>
- $\hat{\mathbf{v}}_p=\mathbf{v}_p^n+\frac{q_p \Delta t}{2 m_p} \mathbf{E}_p^{n+\theta}$ <br/>
- Update Velocity $\overline{\mathbf{v}}_p= R(\hat{\mathbf{v}}_p)$<br/>

with that update $\overline{v_{i}}$ and  evolve:
$$
\begin{align}
 \overline{x^{n}}_{i+1}=x^{n}_{0}+1/2 \cdot \overline{v_{i}} \cdot \Delta t
\end{align}
$$
 Iterate with Particles $x^{n}_{i+1}$ <br/>
 #### Update Fields
 $$
\begin{aligned}
& \mathbf{B}^{n+1}=\mathbf{B}^n-c \Delta t \nabla \times \mathbf{E}^{n+\theta} \\
& \mathbf{E}^{n+1}=\mathbf{E}^n+c \Delta t\left(\nabla \times \mathbf{B}^{n+\theta}-\frac{4 \pi}{c} \mathbf{J}^{n+\theta}\right)\\
& \mathbf{E}^{n+1}=\mathbf{E}^n+ 2 \cdot \mathbf{E}^{n+\theta}
\end{aligned}
$$
 **Code Implenetation Side Node** <br/>
Do 1 Iteration to save Helper Variable. $\overline{x^{n}}$ <br/>
Do 2 Loops with this Helper Variable and overwrite Each Time <br/>
After calculat last time velocity with helper and do full step to i+1 

# Incredients for $E^{n+\theta}$

$$
\begin{align}
A \cdot \mathbf{E}^{n+\theta} & =(\mathbf{I} + \mu) \cdot \mathbf{E}^{n+\theta} - (c\theta\Delta t)^2 \left[ \nabla^2 \mathbf{E}^{n+\theta} + \nabla \nabla \cdot (\mu \cdot \mathbf{E}^{n+\theta}) \right] \\
b & =\mathbf{E}^n + c\theta\Delta t \left( \nabla \times \mathbf{B}^n - \frac{4\pi}{c} \hat{\mathbf{J}}^n \right) - (c\theta\Delta t)^2 4\pi \nabla \hat{\rho}^n
\end{align}
$$
- **Implementierung aller Ableitungen**
    - $\nabla^2$ (Laplace) Vektorfeld F: Fi nach $x^2,y^2,z^2$ abgeleited
    - $\nabla \nabla$ (Grad(Div($\boldsymbol{\cdot}$)) Summe aus 
    - $\nabla \times$ (Curl)
    - $\nabla$ (Divergence)
- Alle Variabeln für b $\hat{\mathbf{J}}^n$ und $\hat{\rho}^n$ <br/>
- **Mu** Def:
$$\mu =4 \pi \theta \Delta t \beta_s \rho_s \sum_p \mathbf{R}_p (\bullet) $$

$$
\mathbf{R}_p (\bullet )=\frac{\bullet+\frac{\beta_p}{c} \bullet \times \mathbf{B}_p^n+\frac{\beta_p^2}{c^2}(\bullet \cdot \mathbf{B}_p^n ) \cdot \mathbf{B}_p^n}{1+\frac{\beta_p^2}{c^2}\left|B_p^n\right|^2} 
$$

Notiz :
$$
\text{grad}(\text{div}(\mathbf{F})) = \nabla (\nabla \cdot \mathbf{F}) = \left( \frac{\partial}{\partial x} (\nabla \cdot \mathbf{F}), \frac{\partial}{\partial y} (\nabla \cdot \mathbf{F}), \frac{\partial}{\partial z} (\nabla \cdot \mathbf{F}) \right).
$$
#### Getting Rho_Hat. Normal Charge Deposition and than get $\hat{J}^{n}$
$$
\begin{align}
 \hat{\rho}^{n}=\rho^{n}-\theta \Delta t \nabla \cdot \hat{J}^{n}
\end{align}
$$

#### Getting J_Hat 
$$
\hat{\mathbf{J}}^n=\sum_p q_p R(\mathbf{v}_p)  S\left(\mathbf{x}-\mathbf{x}^n_p\right)-\theta \Delta t \nabla  \sum_p q_p R(\mathbf{v}_p) R(\mathbf{v}_p) S\left(\mathbf{x}-\mathbf{x}^n_p\right)
$$
J und $\rho$ bestehen sind beides Arrays dh J[index] entspricht einer Pos x. Shae FUnction sind von den PArtikeln


## Using now Solver to Get E_theta


## Deposition From Particles to Grid and vise versa for Later Fields
- **Iter** Particle <br/>
    - Take Pos X,Y,Z Devide my d(x,y,z)
    - Floor it (int())
    - Calcalate all 8 Volume if +1  Volume Distribution and Add them to Gridpoints (-1)^m*(x-xp)*(y-yp)*(z-zp) **Wrong Potentially**
    -  do 1d each and multiplay easier
    -  use Reaptive MApping Modulo <br/>

From Code PIC II ion background to neutralize:                   **Not Implemented** <br/>
- rho -= NP/mx 
- rho *= 2*NPpCell*charge/dx
**Also for Current**




**Sanity Check (Energy Balance):**
- $\mathbf{Electric Energy} \quad 0.5 \sum (Efield^2)*dx$
- $\mathbf{Total Engery}$
- $\mathbf{Kinetic Energy} \quad 0.5*\rho_{weight} \sum (vp^2)*dx$ <br/>
**t=t+dt**

# Give Me some Space to Breath
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
Tack

# Derivation Intermediate Velocity
### Particle Mover (Semi-Implicit)  in CGS: Gauss System (SI without c) c =29,979,245,800 cm/s
Update particle positions:
        $$
          \vec{x}^{n+1} = \vec{x}^n +  \vec{v}^{n+0.5} \Delta t
        $$
Update particle velocities with a semi-implicit scheme:
        $$
             \frac{\vec{v}^{n+1} - \vec{v}^n}{\Delta t} = \frac{q}{m}\left( \vec{E}^{n+\theta} + \frac{\vec{v}^{n+0.5} }{c} \times \vec{B} \right) \quad \vec{v}^{n+0.5} = \frac{\vec{x}^{n+1} + \vec{v}^{n} }{ 2}
        $$
To Solve Equation First Write the Evolving Velocity scheme in Terms of intermetiate step $\vec{v}^{n+0.5}$. Use the Identity of $(\vec{v} x \vec{B} )x \vec{B}$ Compute the Cross Product of $\vec{v}^{n+0.5}$ and B and use the Equation again so it only depends linear on $\vec{v}^{n+0.5}$ tahn solve for it that gives:
$$
\hat{\mathbf{v}}_p=\mathbf{v}_p^n+\frac{q_p \Delta t}{2 m_p} \mathbf{E}_p^{n+\theta}
$$
we get a formula for the intermediate velocity:
$$
\overline{\mathbf{v}}_p=\frac{\hat{\mathbf{v}}_p+\frac{q_p \Delta t}{2 m_p c}\left(\hat{\mathbf{v}}_p \times \mathbf{B}_p^n+\frac{q_p \Delta t}{2 m_p c}\left(\hat{\mathbf{v}}_p \cdot \mathbf{B}_p^n\right) \mathbf{B}_p^n\right)}{1+\frac{q_p^2 \Delta t^2}{4 m_p^2 c^2}\left|B_p^n\right|^2}
$$

# Derivation of Linearized Electromagnetic Field Equations

Starting with the given time-stepping equations for the electromagnetic fields:
$$
\begin{aligned}
& \mathbf{B}^{n+1} = \mathbf{B}^n - c \Delta t \nabla \times \mathbf{E}^{n+\theta}, \\
& \mathbf{E}^{n+1} = \mathbf{E}^n + c \Delta t \left( \nabla \times \mathbf{B}^{n+\theta} - \frac{4 \pi}{c} \mathbf{J}^{n+\theta} \right),
\end{aligned}
$$
we apply the linear interpolation for the fields at the intermediate time step $n+\theta$:
 $\mathbf{E}$, $\mathbf{B}$. Substituting into the equations, we get:
$$
\begin{aligned}
& \mathbf{E}^{n+\theta} = (1 - \theta) \mathbf{E}^n + \theta \mathbf{E}^{n+1}, \\
& \mathbf{B}^{n+\theta} = (1 - \theta) \mathbf{B}^n + \theta \mathbf{B}^{n+1}, \\
\end{aligned}
$$
Than we plug in
$$
\begin{aligned}
& \mathbf{E}^{n+\theta} = (1 - \theta) \mathbf{E}^n + \theta \mathbf{E}^n +\theta c \Delta t \left( \nabla \times \mathbf{B}^{n+\theta} - \frac{4 \pi}{c} \mathbf{J}^{n+\theta} \right) \, [5]\\
& \mathbf{B}^{n+\theta} = (1 - \theta) \mathbf{B}^n + \theta \mathbf{B}^n - \theta c \Delta t \nabla \times \mathbf{E}^{n+\theta} \\
\end{aligned}
$$
2 Terms Cancel $f^{n}*\theta$. Use Identity of $\Delta \times ( \Delta \times E)$ after aplaying Curl on B and plug in curl of B in Equation 5. This results in equation for $E^{n+\theta}$.
$$
\begin{equation}
\mathbf{E}^{n+\theta} - (c\theta\Delta t)^2 \nabla^2 \mathbf{E}^{n+\theta} =
\mathbf{E}^n + c\theta\Delta t \left( \nabla \times \mathbf{B}^n - \frac{4\pi}{c} \mathbf{J}^{n+\theta} \right) - (c\theta\Delta t)^2 4\pi \nabla \rho^{n+\theta}
\end{equation}
$$
Für $\rho$ Tailor Expansion in Time:
$$
\begin{align}
\rho^{n+\theta}&=\rho(x^{n+\theta})=\rho^{n}+(x^{n+\theta}-x^{n})\nabla S(x-x^{n}).. \quad \quad (x^{n+\theta}-x^{n})=-\theta \nabla t \overline{\mathbf{v}}_p \\
\rho^{n+\theta}&=\rho^{n}-\theta \Delta t \nabla \cdot J^{n+\theta}
\end{align}
$$


We can use the same approximation for $\mathbf{J}^{n+\theta}$
$$
\mathbf{J}^{n+\theta}=\sum_p q_p \overline{\mathbf{v}}_p S\left(\mathbf{x}-\mathbf{x}^n\right)-\theta \Delta t \nabla \cdot \sum_p q_p \overline{\mathbf{v}}_p \overline{\mathbf{v}}_p S\left(\mathbf{x}-\mathbf{x}^n\right)
$$
which, rewriting (9) as
$$
\begin{array}{lll}
\overline{\mathbf{v}}_p=\tilde{\mathbf{v}}_p+\beta_p \tilde{\mathbf{E}}^{n+\theta} & \beta_p=\frac{q_p \Delta t}{2 m_p} & \mu \cdot=\sum_s \mu_s \\
\tilde{\mathbf{v}}_p=\mathbf{R}_p \cdot \mathbf{v}_p & \tilde{\mathbf{E}}_p^{n+\theta}=\mathbf{R}_p \cdot \mathbf{E}^{n+\theta} & \mu_{s }\cdot=4 \pi \theta \Delta t \beta_s \rho_s \sum_p \mathbf{R}_p \\
\mathbf{R}_p \cdot=\frac{\mathbf{I}-\frac{\beta_p}{c} \mathbf{I} \times \mathbf{B}_p^n+\frac{\beta_p^2}{c^2} \mathbf{B}_p^n \cdot \mathbf{B}_p^n}{1+\frac{\beta_p^2}{c^2}\left|B_p^n\right|^2} & &
\end{array}
$$
and with
$$
\hat{\mathbf{J}}^n=\sum_p q_p \tilde{\mathbf{v}}_p  S\left(\mathbf{x}-\mathbf{x}^n\right)-\theta \Delta t \nabla \cdot \sum_p q_p \tilde{\mathbf{v}}_p \tilde{\mathbf{v}}_p S\left(\mathbf{x}-\mathbf{x}^n\right)
$$
plucking in FIrst order also 0
$$
\mathbf{J}^{n+\theta}=\hat{\mathbf{J}}^n+\sum_p q_p  \beta_p \tilde{\mathbf{E}}^{n+\theta}S\left(\mathbf{x}-\mathbf{x}^n\right)-\theta \Delta t \nabla \cdot \sum_p q_p \beta_p \tilde{\mathbf{E}}^{n+\theta} \tilde{\mathbf{v}}_p S\left(\mathbf{x}-\mathbf{x}^n\right)+O((\beta_p \tilde{\mathbf{E}}^{n+\theta})^2)
$$
gives
$$
\mathbf{J}^{n+\theta}=\hat{\mathbf{J}}^n+\frac{1}{4 \pi \theta \Delta t} \mu \cdot \mathbf{E}^{n+\theta}
$$

# Maxwell Equation
$$
\begin{aligned}
&\text{Gauss's law for electricity:} && \nabla \cdot \mathbf{E} = 4\pi \rho \\
&\text{Gauss's law for magnetism:} && \nabla \cdot \mathbf{B} = 0 \\
&\text{Faraday's law of induction:} && \nabla \times \mathbf{E} = -\frac{1}{c} \frac{\partial \mathbf{B}}{\partial t} \\
&\text{Ampère's law (with Maxwell's correction):} && \nabla \times \mathbf{B} = \frac{1}{c} \frac{\partial \mathbf{E}}{\partial t} + \frac{4\pi}{c} \mathbf{J}
\end{aligned}
$$


## 📚 References

**ChatGPT**
- Vu, H. X., & Brackbill, J. U. (1982). A numerical solution method for the two-dimensional magnetohydrodynamic equations. *Journal of Computational Physics*.
- Lapenta, G. (2006). Particle simulations of space weather. *Journal of Computational Physics*.

**Own Literature**
- Ott, Tobias & Pfeiffer, Marcel. (2023). PIC schemes for multi-scale plasma simulations. 10.13009/EUCASS2023-770.


In [3]:
import numpy as np 
arr1 = np.array([[1, 2], [3, 4]])
arr2 = np.array([[5, 6], [7, 8]])
"""
Concatenate Verknüpfen: erst über arr1 dann pber arr2 Iterieren
Achsen bestimmt die Ausgangs Achse arr[axis=0][axis=0] 
Axis 0
Mapping Elemente of arr1[index] und speicher in new[index] => Einfach aneinanderhängen
Axis 1
Mapping Elemente of arr1[:][index] und speicher in new[:][index] => Elemente werden nebeinander ANgehängt 
"""
arr = np.concatenate((arr1, arr2), axis=1)
print(arr)
arr1 = np.array([1, 2, 3])
arr2 = np.array([4, 5, 6])
""""
Stacking fügt einfach nur anneinander und zusammenzufügen 
Axis 0
Takes arr1 und put it  in new[index] => Zusammenspeichern getrennt
Axis 1
Mapping Elemente of arr1 und speicher in new[index][0] für array2 index=1 
"""
arr = np.stack((arr1, arr2), axis=1)
print(arr)

[[1 2 5 6]
 [3 4 7 8]]
[[1 4]
 [2 5]
 [3 6]]


In [4]:
arr1 = np.zeros([3,5])
print(np.shape(arr1))
arr1[:,2]=[1,1,1]
arr1[:,3]=[7,0,7]
result = np.sum(arr1**2, axis=0)  # Summe der Quadrate entlang axis=0
print(np.shape(result))  # Ausgabe: (5,)
print(result)

(3, 5)
(5,)
[ 0.  0.  3. 98.  0.]


In [12]:

Nx= (4,2)
Ny=Nz=1
rho = np.zeros([Nx, Ny, Nz])
#print(rho)
print(list(Nx))

TypeError: 'int' object is not iterable